In [175]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

In [176]:
import torch
print("GPU доступний:", torch.cuda.is_available())
print("Назва GPU:", torch.cuda.get_device_name(0))

GPU доступний: True
Назва GPU: Quadro P2000


In [177]:

print(faiss.get_num_gpus())

1


In [178]:
np.random.seed(2025)
pd.options.display.float_format = '{:,.2f}'.format

In [179]:
metadata = pd.read_csv('data/movies_metadata.csv', low_memory=False)

In [180]:
metadata

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,"373,554,033.00",81.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.70,"5,415.00"
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,"262,797,249.00",104.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.90,"2,413.00"
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.00,101.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.50,92.00
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,"81,452,156.00",127.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.10,34.00
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,"76,578,911.00",106.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.70,173.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.00,90.00,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.00,1.00
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.00,360.00,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.00,3.00
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.00,90.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.80,6.00
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.00,87.00,[],Released,NaN,Satan Triumphant,False,0.00,0.00


In [181]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [182]:
metadata.describe()

,revenue,runtime,vote_average,vote_count
count,"45,460.00","45,203.00","45,460.00","45,460.00"
mean,"11,209,348.54",94.13,5.62,109.90
std,"64,332,246.74",38.41,1.92,491.31
min,0.00,0.00,0.00,0.00
25%,0.00,85.00,5.00,3.00
50%,0.00,95.00,6.00,10.00
75%,0.00,107.00,6.80,34.00
max,"2,787,965,087.00","1,256.00",10.00,"14,075.00"


In [183]:
metadata.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [184]:
metadata.duplicated().sum()

17

In [185]:
metadata = metadata.drop_duplicates()

In [186]:
def WR(df, m_quantile=0.90):
    df = df.copy()
    m = df['vote_count'].quantile(m_quantile)
    metadata_clipped = df[df['vote_count'] >= m]
    C = df['vote_average'].mean()
    R = metadata_clipped['vote_average']
    v = metadata_clipped['vote_count']

    df['WR'] = (v / (v + m) * R) + (m / (m + v) * C)
    return df


metadata = WR(metadata)

In [187]:
metadata.sort_values(by='WR', ascending=False)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,WR
314,False,NaN,25000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,...,"28,341,469.00",142.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.50,"8,358.00",8.45
834,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,"245,066,411.00",175.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.50,"6,024.00",8.43
10309,False,NaN,13200000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,19404,tt0112870,hi,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",...,"100,000,000.00",190.00,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Come... Fall In Love,Dilwale Dulhania Le Jayenge,False,9.10,661.00,8.42
12481,False,"{'id': 263, 'name': 'The Dark Knight Collectio...",185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://thedarkknight.warnerbros.com/dvdsite/,155,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,...,"1,004,558,444.00",152.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.30,"12,269.00",8.27
2843,False,NaN,63000000,"[{'id': 18, 'name': 'Drama'}]",http://www.foxmovies.com/movies/fight-club,550,tt0137523,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,...,"100,853,753.00",139.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Mischief. Mayhem. Soap.,Fight Club,False,8.30,"9,678.00",8.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,0.00,90.00,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.00,1.00,NaN
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,0.00,360.00,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.00,3.00,NaN
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,0.00,90.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.80,6.00,NaN
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,0.00,87.00,[],Released,NaN,Satan Triumphant,False,0.00,0.00,NaN


In [188]:
metadata['overview'] = metadata['overview'].astype(str)

In [189]:
metadata[metadata['overview'].apply(lambda x: not isinstance(x, str))]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,WR


In [190]:
metadata = metadata.drop_duplicates(subset='title', keep='first')

In [191]:
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'WR'],
      dtype='object')

In [192]:
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')


embeddings = model.encode(
    metadata['overview'].tolist(),
    normalize_embeddings=True,
    show_progress_bar=True
)

Batches:   0%|          | 0/1322 [00:00<?, ?it/s]

In [193]:
metadata[metadata['title'].duplicated() == True]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,WR


In [194]:
embeddings

array([[ 0.06343901,  0.00102685,  0.09321016, ...,  0.01535521,
         0.04463216,  0.02202344],
       [ 0.08630583,  0.0446149 , -0.04049636, ..., -0.00329479,
        -0.02927732, -0.02655018],
       [-0.10087598,  0.03744174, -0.00092461, ...,  0.05683169,
        -0.02618018,  0.01833552],
       ...,
       [ 0.01217238, -0.0156133 , -0.01670896, ...,  0.04606612,
         0.05611589,  0.01561825],
       [-0.00971064,  0.10046431, -0.06151155, ...,  0.00886031,
         0.00977876, -0.03758177],
       [-0.03125675,  0.01159672, -0.07970025, ..., -0.06735129,
        -0.013464  , -0.11868092]], dtype=float32)

In [195]:
embedding_dim = embeddings.shape[1]

index = faiss.IndexHNSWFlat(embedding_dim, 32)
index.hnsw.efConstruction = 200
index.hnsw.efSearch = 50

index.add(np.array(embeddings, dtype=np.float32))

In [196]:
def find_similar_movies(title, metadata, embeddings, index, model, top_k=10):
    row = metadata[metadata['title'] == title]

    if row.empty:
        return f"No movie with title '{title}' found."

    overview = row.iloc[0]['overview']

    query_embedding = model.encode(overview, normalize_embeddings=True)
    query_embedding = np.array([query_embedding], dtype=np.float32)

    distances, indices = index.search(query_embedding, top_k + 1)

    similar_indices = indices[0][1:]
    similar_titles = metadata.iloc[similar_indices]['title'].tolist()

    return similar_titles

In [197]:
similar = find_similar_movies("The Matrix", metadata, embeddings, index, model)
print(similar)

['A Detective Story', '23', 'Algorithm', 'Commando', 'The Matrix Reloaded', 'Sneakers', 'Double Down', "Kid's Story", 'The Matrix Revisited', 'Nightmare Code']


In [198]:
metadata[metadata['title'] == 'The Matrix']['overview'].values[0]

'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.'

In [199]:
for i in similar:
    print(f"Name: {i} ||| Overview: ",metadata[metadata['title'] == i]['overview'].values[0], '\n\n')

Name: A Detective Story ||| Overview:  A private detective is hired to find the elusive computer hacker "Trinity." Part of the Animatrix collection of animated shorts in the Matrix universe. 


Name: 23 ||| Overview:  The movie's plot is based on the true story of a group of young computer hackers from Hannover, Germany. In the late 1980s the orphaned Karl Koch invests his heritage in a flat and a home computer. At first he dials up to bulletin boards to discuss conspiracy theories inspired by his favorite novel, R.A. Wilson's "Illuminatus", but soon he and his friend David start breaking into government and military computers. Pepe, one of Karl's rather criminal acquaintances senses that there is money in computer cracking - he travels to east Berlin and tries to contact the KGB. 


Name: Algorithm ||| Overview:  A freelance computer hacker discovers a mysterious government computer program. He breaks into the program and is thrust into a revolution. 


Name: Commando ||| Overview:  J